# Wind farm

In [5]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db
from pydae.bmapu import bmapu_builder
import pandapower as pp

In [6]:
zeta = 0.1 # 1.0/np.sqrt(2) 
H_v = 4.0 
WB = 2 *np.pi* 50;
R_v = 0.0
X_v = 0.3

Lt = X_v 
P_max = 1/Lt
fn = np.sqrt(WB*P_max/(2*H_v))/(2*np.pi)

Dp = 0;
K_i = (2*np.pi*fn)**2/(WB*P_max);
K_g = Dp*K_i;
K_p = (2*zeta*2*np.pi*fn - K_g)/(WB*P_max);

T_q = 1.0/(2*np.pi*10/2)
K_q = (1.0 - 0.0)/(1.05 - 0.95)
K_i_q = 1e-6

theta_red = 3.0/180*np.pi
V = 1.0
p_ref = 0.9
q_ref = 0.434616
v_ref = 1.0
T_q = 1.0/(2*np.pi*10/2)

In [10]:
SB = 100
#---------------------
SBwf1 = 75/2/2;
SBwf2 = 50/2;
#---------------------
SBwf  = 4*SBwf1+4*SBwf2;
SBred = 4*SBwf;
#---------------------
Xt = 0.0552; 
Zc = (0.02 + 1j*0.0248);
Xtp = 2*0.06 / (SBwf/100);
Zg = (1/10+1j) / (SBred/100); 
U_b_hv = 66e3
S_b = 100e6
Z_b_hv = U_b_hv**2/S_b

r_ohm_per_km_g = Zg.real*Z_b_hv
x_ohm_per_km_g = Zg.imag*Z_b_hv 
c_nf_per_km_g = 0.0
#---------------------
# Transformadores de los Generadores
#---------------------
Xt1 = Xt / (SBwf1/100);
Xt2 = Xt / (SBwf2/100);
#---------------------
# Colector
#---------------------
Zc1 = 1.0*Zc / (SBwf1/100); # pu, base del sistema
Zc2 = 1.0*Zc / (SBwf2/100);
U_b_mv = 20e3
S_b = 100e6
Z_b_mv = U_b_mv**2/S_b

r_ohm_per_km_c1 = Zc1.real*Z_b_mv
x_ohm_per_km_c1 = Zc1.imag*Z_b_mv 
c_nf_per_km_c1 = 0.0
r_ohm_per_km_c2 = Zc2.real*Z_b_mv
x_ohm_per_km_c2 = Zc2.imag*Z_b_mv 
c_nf_per_km_c2 = 0.0
#---------------------

Rc1 = Zc1.real
Xc1 = Zc1.imag
Rc2 = Zc2.real
Xc2 = Zc2.imag
Rg = Zg.real
Xg = Zg.imag

data = {
    "system":{"name":"wind_farm","S_base":100e6, "K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.001},  
    "lines":[
        {"bus_j": "01", "bus_k":"02", "S_mva":SB, "R_pu":  0.00000, "X_pu": Xt1},      
        {"bus_j": "03", "bus_k":"04", "S_mva":SB, "R_pu":  0.00000, "X_pu": Xt1},      
        {"bus_j": "05", "bus_k":"06", "S_mva":SB, "R_pu":  0.00000, "X_pu": Xt1},      
        {"bus_j": "07", "bus_k":"08", "S_mva":SB, "R_pu":  0.00000, "X_pu": Xt1},      
        {"bus_j": "02", "bus_k":"04", "S_mva":SB, "R_pu":      Rc1, "X_pu": Xc1},     
        {"bus_j": "04", "bus_k":"06", "S_mva":SB, "R_pu":      Rc1, "X_pu": Xc1},     
        {"bus_j": "06", "bus_k":"08", "S_mva":SB, "R_pu":      Rc1, "X_pu": Xc1},     
        {"bus_j": "08", "bus_k":"17", "S_mva":SB, "R_pu":      Rc1, "X_pu": Xc1},     
        {"bus_j": "09", "bus_k":"10", "S_mva":SB, "R_pu":  0.00000, "X_pu": Xt2},        
        {"bus_j": "11", "bus_k":"12", "S_mva":SB, "R_pu":  0.00000, "X_pu": Xt2},        
        {"bus_j": "13", "bus_k":"14", "S_mva":SB, "R_pu":  0.00000, "X_pu": Xt2},        
        {"bus_j": "15", "bus_k":"16", "S_mva":SB, "R_pu":  0.00000, "X_pu": Xt2},        
        {"bus_j": "10", "bus_k":"12", "S_mva":SB, "R_pu":      Rc2, "X_pu": Xc2},     
        {"bus_j": "12", "bus_k":"14", "S_mva":SB, "R_pu":      Rc2, "X_pu": Xc2},     
        {"bus_j": "14", "bus_k":"16", "S_mva":SB, "R_pu":      Rc2, "X_pu": Xc2},     
        {"bus_j": "16", "bus_k":"17", "S_mva":SB, "R_pu":      Rc2, "X_pu": Xc2},     
        {"bus_j": "17", "bus_k":"18", "S_mva":SB, "R_pu":  0.00000, "X_pu": Xtp},       
        {"bus_j": "18", "bus_k":"19", "S_mva":SB, "R_pu":       Rg, "X_pu":  Xg},   
         ],   
    "buses":[
            {"name":  "01", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "02", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "03", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "04", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "05", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "06", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "07", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "08", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "09", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "10", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "11", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "12", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "13", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "14", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "15", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "16", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "17", "P_W": 0.0, "Q_var":  1.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "18", "P_W":-2.5, "Q_var":-0.05, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "19", "P_W": 0.0, "Q_var":  0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
    ],
    "genapes":[{"bus":"19","S_n":100e6,"F_n":50.0,"R_v":0.0,"X_v":0.1,"K_delta":0.001,"K_alpha":1.0}],
    "vscs": [{"bus":"01","type":"vsc_l","S_n":SBwf1*1e6,"F_n":50.0,"X_s":0.05,"R_s":0.005,"K_delta":0.0,
          "ctrl":{"type":"leon_vsg_ll","F_n":50.0,"K_delta":0.0,
                  "R_v":R_v,"X_v":X_v,"K_p":K_p,"K_i":K_i,"K_g":K_g,"K_q":K_q,
                  "T_q":T_q,"K_p_v":1e-6,"K_i_v":1e-6}},
        {"bus":"03","type":"vsc_l","S_n":SBwf1*1e6,"F_n":50.0,"X_s":0.05,"R_s":0.005,"K_delta":0.0,
          "ctrl":{"type":"leon_vsg_ll","F_n":50.0,"K_delta":0.0,
                  "R_v":R_v,"X_v":X_v,"K_p":K_p,"K_i":K_i,"K_g":K_g,"K_q":K_q,
                  "T_q":T_q,"K_p_v":1e-6,"K_i_v":1e-6}},
        {"bus":"05","type":"vsc_l","S_n":SBwf1*1e6,"F_n":50.0,"X_s":0.05,"R_s":0.005,"K_delta":0.0,
          "ctrl":{"type":"leon_vsg_ll","F_n":50.0,"K_delta":0.0,
                  "R_v":R_v,"X_v":X_v,"K_p":K_p,"K_i":K_i,"K_g":K_g,"K_q":K_q,
                  "T_q":T_q,"K_p_v":1e-6,"K_i_v":1e-6}},
        {"bus":"07","type":"vsc_l","S_n":SBwf1*1e6,"F_n":50.0,"X_s":0.05,"R_s":0.005,"K_delta":0.0,
          "ctrl":{"type":"leon_vsg_ll","F_n":50.0,"K_delta":0.0,
                  "R_v":R_v,"X_v":X_v,"K_p":K_p,"K_i":K_i,"K_g":K_g,"K_q":K_q,
                  "T_q":T_q,"K_p_v":1e-6,"K_i_v":1e-6}},
        {"bus":"09","type":"vsc_l","S_n":SBwf2*1e6,"F_n":50.0,"X_s":0.05,"R_s":0.005,"K_delta":0.0,
          "ctrl":{"type":"leon_vsg_ll","F_n":50.0,"K_delta":0.0,
                  "R_v":R_v,"X_v":X_v,"K_p":K_p,"K_i":K_i,"K_g":K_g,"K_q":K_q,
                  "T_q":T_q,"K_p_v":1e-6,"K_i_v":1e-6}},
        {"bus":"11","type":"vsc_l","S_n":SBwf2*1e6,"F_n":50.0,"X_s":0.05,"R_s":0.005,"K_delta":0.0,
          "ctrl":{"type":"leon_vsg_ll","F_n":50.0,"K_delta":0.0,
                  "R_v":R_v,"X_v":X_v,"K_p":K_p,"K_i":K_i,"K_g":K_g,"K_q":K_q,
                  "T_q":T_q,"K_p_v":1e-6,"K_i_v":1e-6}},
        {"bus":"13","type":"vsc_l","S_n":SBwf2*1e6,"F_n":50.0,"X_s":0.05,"R_s":0.005,"K_delta":0.0,
          "ctrl":{"type":"leon_vsg_ll","F_n":50.0,"K_delta":0.0,
                  "R_v":R_v,"X_v":X_v,"K_p":K_p,"K_i":K_i,"K_g":K_g,"K_q":K_q,
                  "T_q":T_q,"K_p_v":1e-6,"K_i_v":1e-6}},
        {"bus":"15","type":"vsc_l","S_n":SBwf2*1e6,"F_n":50.0,"X_s":0.05,"R_s":0.005,"K_delta":0.0,
          "ctrl":{"type":"leon_vsg_ll","F_n":50.0,"K_delta":0.0,
                  "R_v":R_v,"X_v":X_v,"K_p":K_p,"K_i":K_i,"K_g":K_g,"K_q":K_q,
                  "T_q":T_q,"K_p_v":1e-6,"K_i_v":1e-6}}]
}



In [11]:
grid = bmapu_builder.bmapu(data)
#grid.checker()
grid.uz_jacs = True
grid.verbose = False
grid.build('wf8vsgll')

## Validation with PandaPower

In [6]:
net = pp.create_empty_network() 

B01 = pp.create_bus(net,vn_kv=0.69,name='01')
B02 = pp.create_bus(net,vn_kv=20.0,name='02')
B03 = pp.create_bus(net,vn_kv=0.69,name='03')
B04 = pp.create_bus(net,vn_kv=20.0,name='04')
B05 = pp.create_bus(net,vn_kv=0.69,name='05')
B06 = pp.create_bus(net,vn_kv=20.0,name='06')
B07 = pp.create_bus(net,vn_kv=0.69,name='07')
B08 = pp.create_bus(net,vn_kv=20.0,name='08')
B09 = pp.create_bus(net,vn_kv=0.69,name='09')
B10 = pp.create_bus(net,vn_kv=20.0,name='10')
B11 = pp.create_bus(net,vn_kv=0.69,name='11')
B12 = pp.create_bus(net,vn_kv=20.0,name='12')
B13 = pp.create_bus(net,vn_kv=0.69,name='13')
B14 = pp.create_bus(net,vn_kv=20.0,name='14')
B15 = pp.create_bus(net,vn_kv=0.69,name='15')
B16 = pp.create_bus(net,vn_kv=20.0,name='16')
B17 = pp.create_bus(net,vn_kv=20.0,name='17')
B18 = pp.create_bus(net,vn_kv=66.0,name='18')
B19 = pp.create_bus(net,vn_kv=66.0,name='19')

pp.create_transformer_from_parameters(net, lv_bus=B01, hv_bus=B02, sn_mva=SB, vkr_percent=0, vk_percent=100*Xt1,vn_hv_kv=20, vn_lv_kv=0.69, pfe_kw=0, i0_percent=0)      
pp.create_transformer_from_parameters(net, lv_bus=B03, hv_bus=B04, sn_mva=SB, vkr_percent=0, vk_percent=100*Xt1,vn_hv_kv=20, vn_lv_kv=0.69, pfe_kw=0, i0_percent=0)      
pp.create_transformer_from_parameters(net, lv_bus=B05, hv_bus=B06, sn_mva=SB, vkr_percent=0, vk_percent=100*Xt1,vn_hv_kv=20, vn_lv_kv=0.69, pfe_kw=0, i0_percent=0)      
pp.create_transformer_from_parameters(net, lv_bus=B07, hv_bus=B08, sn_mva=SB, vkr_percent=0, vk_percent=100*Xt1,vn_hv_kv=20, vn_lv_kv=0.69, pfe_kw=0, i0_percent=0)      
pp.create_line_from_parameters(net, from_bus= B02, to_bus=B04, r_ohm_per_km=r_ohm_per_km_c1,x_ohm_per_km=x_ohm_per_km_c1,c_nf_per_km = 0.0, length_km=1.0, max_i_ka=1000)     
pp.create_line_from_parameters(net, from_bus= B04, to_bus=B06, r_ohm_per_km=r_ohm_per_km_c1,x_ohm_per_km=x_ohm_per_km_c1,c_nf_per_km = 0.0, length_km=1.0, max_i_ka=1000)    
pp.create_line_from_parameters(net, from_bus= B06, to_bus=B08, r_ohm_per_km=r_ohm_per_km_c1,x_ohm_per_km=x_ohm_per_km_c1,c_nf_per_km = 0.0, length_km=1.0, max_i_ka=1000)    
pp.create_line_from_parameters(net, from_bus= B08, to_bus=B17, r_ohm_per_km=r_ohm_per_km_c1,x_ohm_per_km=x_ohm_per_km_c1,c_nf_per_km = 0.0, length_km=1.0, max_i_ka=1000)    
pp.create_transformer_from_parameters(net, lv_bus=B09, hv_bus=B10, sn_mva=SB, vkr_percent=0, vk_percent=100*Xt2,vn_hv_kv=20, vn_lv_kv=0.69, pfe_kw=0, i0_percent=0)        
pp.create_transformer_from_parameters(net, lv_bus=B11, hv_bus=B12, sn_mva=SB, vkr_percent=0, vk_percent=100*Xt2,vn_hv_kv=20, vn_lv_kv=0.69, pfe_kw=0, i0_percent=0)        
pp.create_transformer_from_parameters(net, lv_bus=B13, hv_bus=B14, sn_mva=SB, vkr_percent=0, vk_percent=100*Xt2,vn_hv_kv=20, vn_lv_kv=0.69, pfe_kw=0, i0_percent=0)        
pp.create_transformer_from_parameters(net, lv_bus=B15, hv_bus=B16, sn_mva=SB, vkr_percent=0, vk_percent=100*Xt2,vn_hv_kv=20, vn_lv_kv=0.69, pfe_kw=0, i0_percent=0)        
pp.create_line_from_parameters(net, from_bus= B10, to_bus=B12, r_ohm_per_km=r_ohm_per_km_c2,x_ohm_per_km=x_ohm_per_km_c2,c_nf_per_km = 0.0, length_km=1.0, max_i_ka=1000),     
pp.create_line_from_parameters(net, from_bus= B12, to_bus=B14, r_ohm_per_km=r_ohm_per_km_c2,x_ohm_per_km=x_ohm_per_km_c2,c_nf_per_km = 0.0, length_km=1.0, max_i_ka=1000),     
pp.create_line_from_parameters(net, from_bus= B14, to_bus=B16, r_ohm_per_km=r_ohm_per_km_c2,x_ohm_per_km=x_ohm_per_km_c2,c_nf_per_km = 0.0, length_km=1.0, max_i_ka=1000),     
pp.create_line_from_parameters(net, from_bus= B16, to_bus=B17, r_ohm_per_km=r_ohm_per_km_c2,x_ohm_per_km=x_ohm_per_km_c2,c_nf_per_km = 0.0, length_km=1.0, max_i_ka=1000),     
pp.create_transformer_from_parameters(net, lv_bus = B17, hv_bus=B18, sn_mva=SB,vkr_percent=0, vk_percent=100*Xtp,vn_hv_kv=66, vn_lv_kv=20, pfe_kw=0, i0_percent=0)       
pp.create_line_from_parameters(net, from_bus= B18, to_bus=B19, r_ohm_per_km=r_ohm_per_km_g,x_ohm_per_km=x_ohm_per_km_g,c_nf_per_km = 0.0, length_km=1.0, max_i_ka=1000),   
    
pp.create_ext_grid(net, bus=B19, vm_pu=1.02, name="Grid", vn_kv=66)

pp.create_gen(net, bus=B01, p_mw=15, vm_pu=1.0, vn_kv=0.69)
pp.create_gen(net, bus=B03, p_mw=15, vm_pu=1.0, vn_kv=0.69)
pp.create_gen(net, bus=B05, p_mw=15, vm_pu=1.0, vn_kv=0.69)
pp.create_gen(net, bus=B07, p_mw=15, vm_pu=1.0, vn_kv=0.69)
pp.create_gen(net, bus=B09, p_mw=20, vm_pu=1.0, vn_kv=0.69)
pp.create_gen(net, bus=B11, p_mw=20, vm_pu=1.0, vn_kv=0.69)
pp.create_gen(net, bus=B13, p_mw=20, vm_pu=1.0, vn_kv=0.69)
pp.create_gen(net, bus=B15, p_mw=20, vm_pu=1.0, vn_kv=0.69)

pp.create_load(net, bus=B17,   p_mw=0, q_mvar=-100, name="Load")
pp.create_load(net, bus=B18, p_mw=250, q_mvar= 5.0, name="Load")

pp.runpp(net)
pp.to_json(net, "pp_wf8vsgll.json")
net.res_bus

import pandapower.shortcircuit as sc

net.ext_grid["s_sc_max_mva"] = 100
net.ext_grid["rx_max"] = 0.1

#sc.calc_sc(net, case="max", ip=True, r_fault_ohm=2.)



In [45]:
ppnet = pp.from_json("pp_wf8vsgll.json")

ppnet.res_bus[ppnet.bus['name'] == '01']

,vm_pu,va_degree,p_mw,q_mvar
0,1.0,12.359691,-15.0,8.364382


0      True
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
Name: name, dtype: bool